In [1]:
:! cabal install --lib pretty-show

Resolving dependencies...
Up to date

In [6]:
{-# LANGUAGE OverloadedStrings #-}

import Database.SQLite.Simple
import Database.SQLite.Simple.FromRow

data Person = Person {
    personId :: Int,
    firstName :: String,
    lastName :: String,
    age :: Int} deriving (Show)

instance FromRow Person where
    fromRow = Person <$> field <*> field <*> field <*> field


-- Connect to the database (or create a new one if it doesn't exist)
conn <- open "test.db"

-- Create a table (if it doesn't exist)
execute_ conn "CREATE TABLE IF NOT EXISTS person (id INTEGER PRIMARY KEY, first_name TEXT, last_name TEXT, age INT)"

-- Insert some data
execute conn "INSERT INTO person (first_name, last_name, age) VALUES (?, ?, ?)" ("John", "Doe", 30)
execute conn "INSERT INTO person (first_name, last_name, age) VALUES (?, ?, ?)" ("Jane", "Doe", 25)

-- Query the data
people <- query_ conn "SELECT * FROM person" :: IO [Person]

-- Print the results
mapM_ print people

-- Close the connection
close conn


Person {personId = 1, firstName = "John", lastName = "Doe", age = 30}
Person {personId = 2, firstName = "Jane", lastName = "Doe", age = 25}

In [7]:
conn <- open "test.db"

In [8]:
query_ conn "SELECT * FROM person" :: IO [Person]

[Person {personId = 1, firstName = "John", lastName = "Doe", age = 30},Person {personId = 2, firstName = "Jane", lastName = "Doe", age = 25}]

------------------

In [1]:
:load ParseLoc
import Data.Aeson
import qualified Data.ByteString.Lazy as B
import Control.Applicative
import Data.Text.Prettyprint.Doc

input <- B.readFile "data2.json"
let mm = decode input :: Maybe [Location]
(length.unjust) mm

l = unjust mm
l'= map parseLoc l

Failed to load module ParseLoc
Could not load module ‘Data.Vector’
It is a member of the hidden package ‘vector-0.13.0.0’.
You can run ‘:set -package vector’ to expose it.
(Note: this unloads all the modules in the current scope.)
Use -v (or `:set -v` in ghci) to see a list of the files searched for.

: 

In [2]:
{-# LANGUAGE OverloadedStrings #-}
import  Data.Text (Text)
import Database.SQLite.Simple

data Location' = Location'
  { database' :: Text
  , dbdesc' :: Text
  , host' :: Text
  } deriving (Show)


instance FromRow Location' where
    fromRow = Location'
      <$> field <*> field <*> field 

In [3]:
conn <- open "mydatabase.db" -- Replace with your database file name

createTableQuery :: Query
createTableQuery = 
    "CREATE TABLE IF NOT EXISTS locations2 (database TEXT,dbdesc TEXT,host)"

-- Create the table
execute_ conn createTableQuery

close conn

In [19]:
conn <- open "mydatabase.db"
result <- query_ conn "SELECT database, count() FROM locations2 GROUP BY database" :: IO [(Text, Int)]
close conn

  -- Now `result` contains the list of tuples returned by the query
  -- You can use `result` within this `do` block
pPrint result

[ ( "A" , 2 ) , ( "Database1" , 1 ) ]

In [17]:
import Text.Show.Pretty (pPrint,ppShow )

In [6]:
conn <- open "mydatabase.db" 
query_ conn "SELECT name FROM sqlite_master WHERE type='table'" :: IO [Only Text]
close conn

[Only {fromOnly = "person"},Only {fromOnly = "locations"},Only {fromOnly = "locations2"}]

In [16]:
instance ToRow Location' where
  toRow (Location' database desc host) =
    toRow (database, desc, host)

In [17]:
insertQuery :: Query
insertQuery = "INSERT INTO locations2 (database, dbdesc, host) VALUES (?, ?, ?)" 

conn <- open "mydatabase.db"

execute conn insertQuery l_record

close conn

In [18]:
let l_record = Location'
          { database' = "Database1"
          , dbdesc' = "Description1"
          , host' = "Host1"
          }
          
l_record



Location' {database' = "Database1", dbdesc' = "Description1", host' = "Host1"}